# Work pool, worker management in Prefect

## 1. Introduction of Work pool and worker

`Worker and Wook Pool` are the most fundamental architectural concepts in `Prefect 3`.

### 1.1 Work pool

In `Prefect 3`, a **Work Pool** represents A `logical namespace` in the Prefect Server. It defines infrastructure type (e.g. process, docker, kubernetes, etc.) and global policies like `concurrency limit`.

> It can hold one or more `work queues` that describe job priorities. I don't think the `work queues` are very useful for Constances.

### 1.2 Worker

The actual `task execution agent` (e.g. process, container, etc.) of the workflow. It connects to **exactly one work pool**. It continuously polls scheduled flows from the connected work pool and executes each flow with the provided environment(i.e. python virtual env).


### 1.3 Worker VS Work Pool

A `worker` must belong to one and only one `work pool`.

A `work pool`:
- can have one or multiple attached `workers`.
- can have one or multiple `work queue`

Below is an example. A work-pool has three work queues, and three workers are attached to the work-pool

```text
(pliu-pool)
 ├── queue: daily-etl
 ├── queue: export
 └── queue: validation
        ▲
        │
 ┌────────────────────────────────────┐
 │  Worker A (process type, limit=1) │
 │  Worker B (process type, limit=1) │
 │  Worker C (process type, limit=1) │
 └────────────────────────────────────┘
```




## 2. Work pool management

We can manage the `Prefect work pools` by using the `web UI, CLI, Terraform, or REST API`. In this tutorial, I will only show how to do it via the `CLI`. Because it's the best way to share and document how to manage work pool.

Below shows commands show how to use CLI to manage Work Pool life-cycle(e.g. create, monitor, pause, resume, delete).

```powershell
# The general form of the CLI for managing work pool is
prefect work-pool [OPTIONS] COMMAND [ARGS]...

# you can get all possible commands with
prefect work-pool --help


# show exiting work pool
prefect work-pool ls

# you should see something like
                                    Work Pools
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Name       ┃ Type    ┃                                   ID ┃ Concurrency Limit ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ pliu-pool  │ process │ 35ed8ee0-dbe7-424b-91ad-b46f49395f54 │ None              │
│ user3_pool │ process │ 26865923-476b-4fc6-a9fa-3aaf6347b5e2 │ None              │
│ user2_pool │ process │ 2bd75b5f-b5fb-4f82-a940-6db17d09c348 │ None              │
│ user1_pool │ process │ 0957b871-0554-4c04-95b2-c0a3ead78b8e │ None              │
│ local-pool │ process │ 05494dd0-20ce-4116-8bf9-5f557549d7ac │ None              │
└────────────┴─────────┴──────────────────────────────────────┴───────────────────┘

# create a new work pool with your user name
prefect work-pool create "%USERNAME%-pool" --type process

# then you can start a worker and add it to your worker pool
prefect worker start --pool "%USERNAME%-pool"

# inspect a work pool
prefect work-pool inspect 'pliu-pool'

# you should see something like
WorkPool(
    id='35ed8ee0-dbe7-424b-91ad-b46f49395f54',
    created=DateTime(2025, 10, 29, 8, 49, 41, 890635, tzinfo=Timezone('UTC')),
    updated=DateTime(2025, 10, 29, 14, 29, 58, 925000, tzinfo=Timezone('UTC')),
    name='pliu-pool',
    type='process',
    base_job_template={
        'job_configuration': {'command': '{{ command }}', 'env': '{{ env }}', 'labels': '{{ labels }}', 'name': '{{ name }}', 'stream_output': '{{ stream_output }}', 'working_dir': '{{ working_dir }}'},
        'variables': {
            'properties': {
                'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Name given to infrastructure created by a worker.', 'title': 'Name'},
                'env': {'additionalProperties': {'anyOf': [{'type': 'string'}, {'type': 'null'}]}, 'description': 'Environment variables to set when starting a flow run.', 'title': 'Environment Variables', 'type': 'object'},
                'labels': {'additionalProperties': {'type': 'string'}, 'description': 'Labels applied to infrastructure created by a worker.', 'title': 'Labels', 'type': 'object'},
                'command': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'The command to use when starting a flow run. In most cases, this should be left blank and the command will be automatically generated by the worker.', 'title': 'Command'},
                'stream_output': {'default': True, 'description': 'If enabled, workers will stream output from flow run processes to local standard output.', 'title': 'Stream Output', 'type': 'boolean'},
                'working_dir': {
                    'anyOf': [{'format': 'path', 'type': 'string'}, {'type': 'null'}],
                    'default': None,
                    'description': 'If provided, workers will open flow run processes within the specified path as the working directory. Otherwise, a temporary directory will be created.',
                    'title': 'Working Directory'
                }
            },
            'type': 'object'
        }
    },
    status=WorkPoolStatus.READY,
    storage_configuration=WorkPoolStorageConfiguration(),
    default_queue_id='2b7c59fb-39f1-4c24-8821-b99e0663798a'
)

# Preview scheduled work of a work pool in the next 12 hours
prefect work-pool preview 'pliu-pool' --hours 12

# if a workflow is scheduled on this work pool, you should see something like
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scheduled Start Time             ┃ Run ID                               ┃ Name                ┃ Deployment ID                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2025-10-29 16:37:52.950000+00:00 │ 019a309e-c009-7080-a931-7f261a479b2a │ archetypal-crayfish │ f950cdcb-1f39-4380-b114-8656322ef64e │
│ 2025-10-29 17:37:52.950000+00:00 │ 019a309e-c009-7793-af29-c0a9131d63c5 │ petite-nightingale  │ f950cdcb-1f39-4380-b114-8656322ef64e │
│ 2025-10-29 18:37:52.950000+00:00 │ 019a309e-c009-7717-b979-4b849dd57fcf │ curly-bullmastiff   │ f950cdcb-1f39-4380-b114-8656322ef64e │
└──────────────────────────────────┴──────────────────────────────────────┴─────────────────────┴──────────────────────────────────────┘

# control the work pool lifecycle
prefect work-pool pause|resume|delete 'pliu-pool'
```


## 3. Worker management

As we have the `Work Pool` created, we can now attach `Worker` on the created Work Pool.

Below commands shows how to manage Worker life-cycle

```powershell
# the general form is
prefect worker [OPTIONS] COMMAND [ARGS]...

# 1. start a worker with options
prefect worker start -p "pliu-pool" --name "%USERNAME%-worker" --limit 1

# -p means attached the worker to a dedicated work pool.
# --name specify the worker name. Default vaule will be auto generated.
# --limit specify the concurrency limit of the worker. Default value is unlimited


# 2. Stop a worker
# As the worker runs as a process, Use Ctrl + C (SIGINT).
# The worker will:
#   - Finish currently running flows
#   - Stop polling new runs
#   - Shut down gracefully

```

> `--install-policy option` defines policy for code downloads (e.g. always, never, missing). only has effect when you create worker on `Prefect Cloud`. As our prefect server runs on an air-capped server. This option is useless.

> Do not close the terminal where you launch the worker, the worker will be killed after the close of the terminal

After adding workers to a work pool, you can check the work pool panel in the web UI. Below figure is an example of a work pool which contains two worker

![worker_in_work_pool.png](../assets/worker_in_work_pool.png)


## 4. Concurrency for worker and work pool

By default, a Prefect work pool has no concurrency limit. It means if four users start to work at the same time, and each of them will launch 10 workflows. There will be 40 workflows which start simultaneously. These workflows will consume all the resources of the server, and no one can work anymore.

The below figure shows an example of four workflow runs simultaneously.

![multiple_running_workflow.png](../assets/multiple_running_workflow.png)

To avoid this problem, I propose to each user in the Bulle to have:
- One work pool with concurrency limit 1
- One worker attached to the work pool, which has concurrency limit 1.

```powershell
# you can set up concurrency limit of your work pool to 1
prefect work-pool set-concurrency-limit "%USERNAME%-pool" 1

# expected output
Set concurrency limit for work pool 'pliu-pool' to 1

# when you start a worker, always setup limit to 1
prefect worker start -p "%USERNAME%-pool" --name "%USERNAME%-worker" --limit 1
```

```text
                ┌────────────────────────────────────────────┐
                │              Prefect Server                │
                └────────────────────────────────────────────┘
                                  │
                                  │
      ┌──────────────────┬──────────────────┬──────────────────┬──────────────────┐
      │ UserA-pool       │ UserB-pool       │ UserC-pool       │ UserD-pool       │
      │ --limit 1        │ --limit 1        │ --limit 1        │ --limit 1        │
      │ UserA-Worker     │ UserB-Worker     │ UserC-Worker     │ UserD-Worker     │
      │ --limit 1        │ --limit 1        │ --limit 1        │ --limit 1        │
      └──────────────────┴──────────────────┴──────────────────┴──────────────────┘

```

> A global concurrency limit of the prefect server will be set too. Base on your job, I can set it to 1 or 2. The submitted workflow will not be lost, they will be on `pending` state, when a worker is available, the new workflow inline will be executed.

```powershell
# create a global concurrency limit
prefect concurrency-limit create "global_limit" 1

# Delete the concurrency limit:
prefect concurrency-limit delete 'global_limit'

# Inspect the concurrency limit:
prefect concurrency-limit inspect 'global_limit'

# expected output
╭────────────────────────────────────────────────────────────────────────╮
│       Concurrency Limit ID: 56c44251-784f-43ed-b617-8f6124449f02       │
│ ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓ │
│ ┃ Tag          ┃ Concurrency Limit ┃ Created        ┃ Updated        ┃ │
│ ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩ │
│ │ global_limit │ 1                 │ '1 minute ago' │ '1 minute ago' │ │
│ └──────────────┴───────────────────┴────────────────┴────────────────┘ │
│ ┏━━━━━━━━━━━━━━━━━━━━━┓                                                │
│ ┃ Active Task Run IDs ┃                                                │
│ ┡━━━━━━━━━━━━━━━━━━━━━┩                                                │
│ └─────────────────────┘                                                │
╰────────────────────────────────────────────────────────────────────────╯
```